In [9]:
#!pip install nltk
!pip install gensim

     |████████████████████████████████| 24.7MB 7.4MB/s eta 0:00:01     |█████████████████████████████▍  | 22.7MB 7.4MB/s eta 0:00:01     |███████████████████████████████▎| 24.2MB 7.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 9.6MB/s  eta 0:00:01
     |████████████████████████████████| 1.4MB 9.2MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.9.0-cp36-none-any.whl size=73089 sha256=e6fd40f0115194ae473c7a31a9244a65dd9f0e3ce553eecf7820cfdd9f3a56be
  Stored in directory: /Users/nhuang37/Library/Caches/pip/wheels/ab/10/93/5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open


In [3]:
# Import dependencies
import nltk
import json
import io
import gzip
import torch
import string
import random
# import jsonlines
import pandas as pd
import pickle as pkl
import numpy as np
from tqdm import tqdm_notebook
from functools import partial
# import mwparserfromhell

import torch
import torch.nn as nn
import torch.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, RandomSampler, SequentialSampler, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer

In [4]:
PATH_TO_DATA = "//Users/nhuang37/Desktop/NYU DS/Yr 2 Sem1/Inference/Project/"

In [5]:
wiki_df = pkl.load(open(PATH_TO_DATA + "en_outlinks_tokens_df.pkl", "rb"))
wiki_df.head()

,QID,title,raw_outlinks,outlinks,raw_tokens,tokens,mid_level_categories
0,Q6199,Anarchism,"[[[Anti-authoritarianism|anti-authoritarian]],...","[Anti-authoritarianism, political philosophy, ...","[anarchism, is, an, anti, authoritarianism, an...","[anarchism, anti, authoritarianism, anti, auth...","[History_And_Society.History and society, Hist..."
1,Q38404,Autism,"[[[Psychiatry]], [[Interpersonal relationship|...","[Psychiatry, Interpersonal relationship, commu...","[autism, is, developmental, disorder, characte...","[autism, developmental, disorder, characterize...","[STEM.Medicine, STEM.Biology, History_And_Soci..."
2,Q101038,Albedo,[[[File:Albedo-e hg.svg|thumb|upright=1.3|The ...,"[File:Albedo-e hg.svg, diffuse reflection, sun...","[sunlight, relative, to, various, surface, con...","[sunlight, relative, various, surface, conditi...","[STEM.Physics, STEM.Space, History_And_Society..."
3,Q173,Alabama,"[[[Coat of arms of Alabama|Coat of arms]], [[N...","[Coat of arms of Alabama, Northern flicker, Di...","[alabama, alabama, is, nicknamed, the, norther...","[alabama, alabama, nicknamed, northern, flicke...",[Geography.Americas]
4,Q41746,Achilles,[[[File:Achilles fighting against Memnon Leide...,[File:Achilles fighting against Memnon Leiden ...,"[dating, to, , three, zero, zero, bc, of, ach...","[dating, three, zero, zero, achilles, killing,...","[Culture.People, Geography.Europe, History_And..."


In [7]:
#toy example of create prior matrix
wiki_df_sub = wiki_df.head(1000)

In [12]:
import gensim
from gensim.utils import simple_preprocess
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(57)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed
[nltk_data]     (_ssl.c:852)>


In [ ]:
def lemmatize_stemming(list_of_tokens):
    output = []
    for token in list_of_tokens:
        output.append(stemmer.stem(lemmatizer.lemmatize(token, pos='v')))
    return output

In [13]:
dictionary = gensim.corpora.Dictionary(wiki_df_sub.tokens)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=1000)

In [16]:
bow_corpus = [dictionary.doc2bow(doc) for doc in wiki_df_sub.tokens]

In [17]:
len(bow_corpus)

1000

In [18]:
%%timeit
lda_model = gensim.models.LdaMulticore(bow_corpus[:100], num_topics=10, id2word=dictionary, passes=2, workers=2)

1.03 s ± 110 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
# Build LDA model
from gensim.models import LdaModel

lda_model = LdaModel(corpus=bow_corpus[:100],
                       id2word=dictionary,
                       num_topics=10, 
                       random_state=100,
                       update_every=1,
                       chunksize=100,
                       passes=10,
                       alpha='auto', eta='auto', #use guided LDA with prior
                       per_word_topics=True)

In [38]:
# Print the Keyword in the 10 topics
from pprint import pprint

pprint(lda_model.print_topics())

[(0,
  '0.044*"war" + 0.018*"battle" + 0.018*"army" + 0.016*"union" + 0.013*"civil" '
  '+ 0.013*"god" + 0.011*"greek" + 0.010*"british" + 0.008*"son" + '
  '0.008*"military"'),
 (1,
  '0.018*"water" + 0.017*"earth" + 0.015*"metal" + 0.010*"element" + '
  '0.009*"energy" + 0.009*"chemical" + 0.008*"air" + 0.008*"elements" + '
  '0.008*"gas" + 0.008*"oil"'),
 (2,
  '0.033*"art" + 0.018*"human" + 0.014*"social" + 0.011*"science" + '
  '0.009*"society" + 0.008*"research" + 0.008*"cultural" + 0.008*"study" + '
  '0.007*"ancient" + 0.007*"medical"'),
 (3,
  '0.075*"film" + 0.012*"best" + 0.010*"production" + 0.009*"art" + '
  '0.007*"company" + 0.007*"series" + 0.007*"works" + 0.007*"japan" + '
  '0.006*"computer" + 0.006*"york"'),
 (4,
  '0.017*"city" + 0.016*"country" + 0.012*"empire" + 0.011*"population" + '
  '0.009*"language" + 0.009*"government" + 0.009*"area" + 0.009*"sea" + '
  '0.008*"war" + 0.008*"province"'),
 (5,
  '0.028*"british" + 0.025*"water" + 0.022*"species" + 0.020*"land

In [42]:
top_topics = lda_model.top_topics(bow_corpus[:100]) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
num_topics = 10
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

Average topic coherence: -0.7221.
[([(0.016547319, 'city'),
   (0.015866887, 'country'),
   (0.011522571, 'empire'),
   (0.011373947, 'population'),
   (0.009453608, 'language'),
   (0.009380453, 'government'),
   (0.008986137, 'area'),
   (0.008602692, 'sea'),
   (0.008278159, 'war'),
   (0.007957151, 'province'),
   (0.007905441, 'largest'),
   (0.0076013855, 'east'),
   (0.007474602, 'region'),
   (0.006961603, 'roman'),
   (0.0065104775, 'asia'),
   (0.0063733486, 'european'),
   (0.0059433975, 'republic'),
   (0.0057228697, 'river'),
   (0.0056548533, 'central'),
   (0.0054944395, 'established')],
  -0.514093990082144),
 ([(0.07496085, 'film'),
   (0.011824957, 'best'),
   (0.009511555, 'production'),
   (0.009317298, 'art'),
   (0.00733761, 'company'),
   (0.007050407, 'series'),
   (0.007021674, 'works'),
   (0.0068642674, 'japan'),
   (0.0060590324, 'computer'),
   (0.0057499013, 'york'),
   (0.005643558, 'television'),
   (0.005544972, 'began'),
   (0.0053234617, 'released'),


In [ ]:
#max_length = 1000
#create prior matrix
#num_workers